# Notebook for developing and testin the Itinerary Code Pilot

In [ ]:
import requests
import json
import os
import openai
import ls_nb_utils
import pandas as pd
from geopy import distance

In [ ]:
openai.api_key  = os.environ['OPENAI_API_KEY']
client = openai.OpenAI()


In [ ]:
ls_app_url =  'https://pocketverse.herokuapp.com/LS_API'
ls_app_username = os.environ['LS_USERNAME']
ls_app_password = os.environ['LS_PASSWORD']
ls_app_username, ls_app_password

In [ ]:
resp_app = ls_nb_utils.get_access_token(ls_app_username, ls_app_password, 'jwt_login', ls_app_url)
resp_app

In [ ]:
token=resp_app['token']

In [ ]:
%%time
params = {"searchScope": "FRIEND"}
params = {}
params = {"serachScope": "San Francisco"}
res = ls_nb_utils.test_ls_post(params, 'place/search', url = ls_app_url, token = token, target = 'packet')

In [ ]:
places = res['foundPlaces']
len(places)

In [ ]:
places[1]['place']

In [ ]:
places[1]['userStories'][0]

In [ ]:
def make_stories(pl):
    user_stories = pl['userStories']
    location =  pl['place']['locationString']
    user_st_text = f'A number of tourists visited {location} and left their stories.\n'
    for us in user_stories:
        user = us['user']['username']
        story = ''
        for st in us['stories']:
            t = st['timestamp']
            s = st['story']
            if t:
                story += f' on {t} '
            story += s                      
        user_st_text += f'Visitor {user} told the following stories: {story}' + '\n-------------------\n'
    return user_st_text
s = make_stories(places[0])
print(s)

In [ ]:
def select_places(places, value, by = 'user', logic = 'eq' ):
    ret = []
    match by:
        case 'user':
            for pl in places:
                user_stories = pl['userStories']
                for us in user_stories:
                    user = us['user']['username']
                    match logic:
                        case 'eq':
                            if user == value:
                                ret.append(pl)
                        case 'neq':
                            if user != value:
                                ret.append(pl)
                                
        case 'distance':
            orig_lat, orig_long = value['origin']
            orig = (orig_lat, orig_long)
            rad = value['radius_miles']
            for pl in places:
                coor = pl['place']['coordinates']
                lat = coor['lat']
                lng = coor['lng']
                pl_coor = (lat, lng)
                dist = distance.distance(orig, pl_coor).miles
                if (dist <= rad):
                    ret.append(pl)
        case _:
            ret = []
    return ret

sel_places = select_places(places, 'liza', by = 'user', logic = 'eq')    

In [ ]:
len(sel_places), len(places)

In [ ]:
orig_value = {'origin':(37.7548329217169, -122.44777880753426), 'radius_miles':25}
sel_dist = select_places(places, orig_value, by = 'distance', logic = 'le')
len(sel_dist)

In [ ]:
def make_all_stories(places):
    all_stories = ''
    for pl in places:
        all_stories += make_stories(pl)
    return all_stories

user_stories = make_all_stories(sel_dist)
print(user_stories)

## Using Open AI

In [ ]:
# %pip install geopy

In [ ]:
model_name = "gpt-3.5-turbo-1106"
def req_ai0(system_message, report_message, content):
    # app.logger.debug(system_message)
    messages=[
            {"role": "system", "content": system_message},
            {"role": "system", "content": report_message}, 
            {"role": "user", "content": content}
            ]
    response = client.chat.completions.create(
            model=model_name,
            temperature = 0,
            seed = 1234,
            messages=messages
            )
    answer = response.choices[0].message.content    
    # app.logger.debug(answer)
    try:
        ans = json.loads(answer)
    except:
        ans = {'text':answer}
        
    return ans


In [ ]:
def req_ai(system_message, report_message, content):
    # app.logger.debug(system_message)
    messages=[
            {"role": "system", "content": system_message},
            # {"role": "system", "content": report_message}, 
            {"role": "user", "content": content}
            ]
    response = client.chat.completions.create(
            model=model_name,
            temperature = 0,
            seed = 1234,
            messages=messages
            )
    answer = response.choices[0].message.content  

    messages=[
            {"role": "system", "content": 'Restructure the user content as a JSON object. Put itenerary items in the chronological order'}, 
            {"role": "user", "content": answer}]
    response = client.chat.completions.create(
            model=model_name,
            temperature = 0,
            seed = 1234,
            messages=messages
            )
    answer = response.choices[0].message.content     
    # app.logger.debug(answer)
    try:
        ans = json.loads(answer)
    except:
        ans = {'text':answer}
        
    return ans


In [ ]:
system_message = '''You are a travel agent and a scheduler. Prepare an itenerary and a schedule based on the provided content.
If the relevant information is absent in the content, use other sources available to you.
'''
report_message = "Report result as a JSON object."

In [ ]:
place = 'San Francisco'

date = '2024:04:01'
activities = "golf, lunch, dinner, lodging"
content = f'I am going to be in {place} on {date} and want to do the following during this day: {activities}. Content: {user_stories}'

In [ ]:
res = req_ai(system_message, report_message, content)

In [ ]:
import pprint

In [ ]:
pprint.pprint(res)

In [ ]:
print (res)